### This notebook addresses how to convert a model from keras .h5 to tflite and optionally edge-tpu within the context of YoloV3

If you need support for converting darknet to keras, please refer to [this github](https://github.com/qqwweee/keras-yolo3).

In [2]:
import os
import tensorflow as tf

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

model = tf.keras.models.load_model('yolov3-aerial.h5')

### Though keras models tend to support dynamic shapes, many edge devices do not. So we convert the model to a static shape and preprocess the images appropriately before invoking the model during inference.

In [3]:
model_config = model.get_config()

In [4]:
model_config['layers'][0]['config']['batch_input_shape'] = (1, 416, 416, 3)

### Edge-TPU does not support LeakyReLU activation layers. However, during inference we can replace them with the supported PReLU.

Note, PReLU is not a suitable replacement during the training stage so do not train in the Edge-TPU.

In [5]:
# edge-tpu only

for layer in model_config['layers']:
    if layer['class_name'] != 'LeakyReLU':
        continue

    name_index = layer['config']['name'][len('leaky_re_lu_'):]

    layer['class_name'] = 'PReLU'
    layer['config'] = {
        'name': f'p_re_lu_{name_index}',
        'alpha_initializer': {'class_name': 'Constant', 'config': {'value': layer['config']['alpha']}},
        'alpha_regularizer': None,
        'alpha_constraint': None,
        'shared_axes': [1, 2]  # required but not sure if it's correct
    }

In [6]:
new_model = model.__class__.from_config(model_config)

In [7]:
# copy over all weights except for the PReLU layers

weights = [layer.get_weights() for layer in model.layers[1:]]
for layer, weight in zip(new_model.layers[1:], weights):
    if layer.name.startswith('p_re_lu_'):
        continue

    layer.set_weights(weight)

### Compiling the model should automatically calculate the shapes of the internal layers now that we defined the shape of the output layer

In [8]:
new_model.compile()

In [9]:
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image_input (InputLayer)       [(1, 416, 416, 3)]   0           []                               
                                                                                                  
 conv2d (Conv2D)                (1, 416, 416, 32)    864         ['image_input[0][0]']            
                                                                                                  
 batch_normalization (BatchNorm  (1, 416, 416, 32)   128         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 p_re_lu_ (PReLU)               (1, 416, 416, 32)    32          ['batch_normalization[0][0]']

In [10]:
new_model.save('yolov3-aerial-compiled.h5')

### Now that we have a properly formatted keras model, we will convert it to TFLite

There are a few things to note here:
 - We will perform quantization to reduce the size of the model and improve inference speed
 - For CPUs we can use dynamic range quantization which is quite effective
 - For Edge-TPU we must use full integer quantization which may involve some accuracy loss. We are also required to provide a representative dataset to calibrate the quantization process (whereas other devices this is optional and may improve accuracy)
 - For GPUs we must use float16 quantization which is less memory efficient than the alternatives but still lowers memory usage by about 2 times

 For a representative dataset, you need only a couple hundred examples. We will be using the dataset used to fine-tune our aerial model which contains about 130 images.

In [ ]:
# clone repository
!git clone git@github.com:jekhor/aerial-cars-dataset.git

In [11]:
def representative_dataset():
    folder_path = 'aerial-cars-dataset/'
    imgs = [img_path for img_path in os.listdir(folder_path) if img_path.endswith('.jpg') or img_path.endswith('.png')]

    for img_path in imgs:
        img = tf.keras.preprocessing.image.load_img(folder_path + img_path, target_size=(416, 416))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)
        img_array = img_array / 255.0
        yield [img_array]

In [12]:
converter = tf.lite.TFLiteConverter.from_keras_model(new_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
# converter.target_spec.supported_types = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]  # this stil doesn't work for some reason
converter.inference_input_type = tf.uint8  # this is required for edge-tpu
converter.inference_output_type = tf.uint8  # this is required for edge-tpu

In [13]:
tflite_model = converter.convert()

2023-05-17 19:18:59.032548: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,?,?]
	 [[{{node inputs}}]]
2023-05-17 19:18:59.165127: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,?,?]
	 [[{{node inputs}}]]
2023-05-17 19:19:05.190499: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,?,?]
	 [[{{node inputs}}]]
2023-05-17

INFO:tensorflow:Assets written to: /tmp/tmp_oxizn4k/assets


INFO:tensorflow:Assets written to: /tmp/tmp_oxizn4k/assets
/home/allanlago/anaconda3/envs/tf-gpu/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:789: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-17 19:19:16.348892: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-05-17 19:19:16.348930: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-05-17 19:19:16.349798: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp_oxizn4k
2023-05-17 19:19:16.369667: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-17 19:19:16.369702: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp_oxizn4k
2023-05-17 19:19:16.440695: I tensorflow/compile

In [14]:
with open('yolov3-aerial-int8.tflite', 'wb') as f:
  f.write(tflite_model)

In [15]:
# edge-tpu only
!apt install edgetpu_compiler
!edgetpu_compiler yolov3-aerial-int8.tflite

/usr/bin/zsh: /home/allanlago/anaconda3/envs/tf-gpu/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
/usr/bin/zsh: /home/allanlago/anaconda3/envs/tf-gpu/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
/bin/bash: /home/allanlago/anaconda3/envs/tf-gpu/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 5729 ms.

Input model: yolov3-aerial-int8.tflite
Input size: 59.49MiB
Output model: yolov3-aerial-int8_edgetpu.tflite
Output size: 60.06MiB
On-chip memory used for caching model parameters: 6.84MiB
On-chip memory remaining for caching model parameters: 5.75KiB
Off-chip memory used for streaming uncached model p